In [123]:
# aggregate checkouts and filled holds by iso week, and location 
sql = """
WITH transactions as (
	SELECT
	CASE c.op_code
	    WHEN 'o' THEN 'checkouts total'
	    WHEN 'f' THEN 'filled holds'
	END as op_name,
	(
		select
		m.location_code
		from
		sierra_view.statistic_group_myuser as m
		where
		m.code = c.stat_group_code_num
	) as transaction_location_code,
	EXTRACT(isoyear from c.transaction_gmt)::TEXT || '-W' || EXTRACT(week FROM c.transaction_gmt)::TEXT as iso_week,
	count(*) as count,
	count(distinct c.patron_record_id) as distinct_patrons

	FROM
	sierra_view.circ_trans as c

	WHERE
	-- transaction operations: 'checkout' and 'filled hold' 
	c.op_code IN ('o', 'f')
    
	-- transactions with a timestamp from X day ago...
	-- AND date(c.transaction_gmt) = date(NOW() - '%s day'::interval)
	AND ptype_code != '196'

	GROUP BY
	op_name,
	transaction_location_code,
	iso_week
)

-- we want every location that we choose to be represented, even if there is no data
SELECT DISTINCT
m.location_code,
t.iso_week,
coalesce(t.op_name, 'checkouts total') as op_name,
coalesce(t.count, 0) as count,
coalesce(t.distinct_patrons, 0) as distinct_patrons

FROM
sierra_view.statistic_group_myuser as m

LEFT OUTER JOIN
transactions as t
ON
  t.transaction_location_code = m.location_code
  

order by
m.location_code,
op_name,
t.iso_week
"""

# fill our dataframes
df_sierra = pd.read_sql(sql=sql, con=con)
df_local = pd.read_csv('2019-checkouts_filled_holds_by_branch.csv')


FileNotFoundError: [Errno 2] No such file or directory: '2019-checkouts_filled_holds_by_branch.csv'

In [ ]:
df = pd.read_sql(sql=sql, con=con, params=[1])

# filter the dataframe
df = df[df['location_code'].isin(['sm', 'an', '1', 'ba', 'gr', 'ge', 'ha', 'ma', 'dt'])]

df['transaction_date'].fillna(method='ffill', inplace=True)
df['transaction_date'] = pd.to_datetime(df['transaction_date'])

alt.Chart(df).mark_bar().encode(
    y='count',
    column='location_code',
    x=alt.X('op_name', title='', sort='-y'),
    color='op_name',
    tooltip=['transaction_date','location_code', 'op_name', 'count']
).properties(
    title='Total Checkouts VS Filled Holds Per Location',
).configure_title(
    # fontSize=20,
    anchor='start',
)

In [ ]:
# make it vertical!

alt.Chart(df).mark_bar().encode(
    x='count',
    row='location_code',
    y=alt.Y('op_name', title=''),
    color='op_name',
    tooltip=['transaction_date','location_code', 'op_name', 'count']
).properties(
    title='Total Checkouts VS Filled Holds Per Location',
#     height=300
).configure_title(
    # fontSize=20,
    anchor='start',
)

In [ ]:
def create_chart(df, days_ago):
    chart = alt.Chart(df).mark_bar().encode(
        x='count',
        row='location_code',
        y=alt.Y('op_name', title=''),
        color='op_name',
        tooltip=['transaction_date','location_code', 'op_name', 'count']
        ).properties(
            title='{} Days Ago - Total Checkouts VS Filled Holds Per Location'.format(days_ago),
            width=300
        )
    return chart

# create an empty list for our charts 
charts = []

for i in range(1, 4):
    print(i)
    df = pd.read_sql(sql=sql, con=con, params=[i])
    df = df[df['location_code'].isin(['sm', 'an', '1', 'ba', 'gr', 'ge', 'ha', 'ma', 'dt'])]

    df['transaction_date'].fillna(method='ffill', inplace=True)
    df['transaction_date'] = pd.to_datetime(df['transaction_date'])
    
    chart = create_chart(df, i)
    charts.append(chart)
    
print('done!')

alt.hconcat(charts[0],
            charts[1],
            charts[2],
           ).configure_title(
        # fontSize=20,
        anchor='start',
    )

In [ ]:
# print(df_sierra[df_sierra['location_code'] == '1'])
# # print(df)

In [ ]:

def create_chart(src_loc, df_sierra, df_local):
    """
    returns a 2x2 chart
    """
    
    charts = []
    
    # produce the top chart
    source = df_sierra[df_sierra['location_code'] == src_loc] 
    chart1 = alt.Chart(source).mark_line(strokeWidth=3).encode(
        x='iso_week:O',
        y='count',
        color='op_name',
    ).properties(
        title='{} - Checkouts and Filled Holds (Items)'.format(src_loc),
        width=300)


    chart2 = alt.Chart(source).mark_line(strokeWidth=3).encode(
        x='iso_week:O',
        y='distinct_patrons',
        color='op_name',
    ).properties(
        title='{} - Checkouts and Filled Holds (Patrons)'.format(src_loc),  
        width=300
    )

    charts.append(alt.hconcat(chart1, chart2))
    
    
    # produce the bot chart
    source = df_local[ (df_local['location_code'] == src_loc) & (df_local['iso_week'] >= '2019W-21') & (df_local['iso_week'] <= '2019W-31') ]
    
    chart1 = alt.Chart(source).mark_line(strokeWidth=3).encode(
        x='iso_week:O',
        y='count',
        color='op_name',
    ).properties(
        title='{} - Checkouts and Filled Holds (Items)'.format(src_loc),
        width=300
    )

    chart2 = alt.Chart(source).mark_line(strokeWidth=3).encode(
        x='iso_week:O',
        y='distinct_patrons',
        color='op_name',
    ).properties(
        title='{} - Checkouts and Filled Holds (Patrons)'.format(src_loc),  
        width=300
    )

    charts.append(alt.hconcat(chart1, chart2))
    
    return alt.vconcat(*charts)


In [ ]:
# comment lines to remove which locations we want
locations = [
    '1',
    'av',
    'cr',
    'ha',
#     '1w',
    'gr',
#     'grw',
    'lv',
    'ww',
    'wy',
#     'haw',
    'hp',
    'mm',
    'mn',
    'ma',
    'md',
    'an',
    'mt',
    'mw',
    'ba',
    'mo',
    'nw',
    'ch',
    'ns',
    'bh',
    'cl',
    'nr',
    'pl',
    'co',
#     'cvw',
    'os',
    'cv',
    'oa',
    'dp',
#     'cll',
    'ep',
    're',
    'dt',
    'pr',
    'wt',
    'fo',
    'wh',
    'sm',
    'gh',
    'sh',
    'ge',
    'sb',
#     'rew',
#     '1l',
#     '51',
#     'dc',
    'oal',
#     'anw'
]

In [ ]:
# create_chart('an', df_sierra, df_local)
# create_chart('ba', df_sierra, df_local)

locations.sort()

for location in locations:
    print(location, end=', ')
    chart = create_chart(location, df_sierra, df_local)
    chart.save('{}.png'.format(location))

print('\ndone!')
# chart = create_chart('an', df_sierra, df_local)
# chart.save('an.png')

In [ ]:
# locations.sort()
locations

In [ ]:
# find patrons (non-expired) with last activity date within 3 years, and aggregate the counts...  
sql = '''
with patrons as (
	select
	*
	from
	sierra_view.patron_record as p
	join
	sierra_view.record_metadata as r
	on
	  r.id = p.record_id
	  and r.campus_code = ''
	  
	where
	expiration_date_gmt > NOW()
)

select
NOw()::date - activity_gmt::date as last_active_days,
count(*)

from
patrons

where
NOw()::date - activity_gmt::date is not null
AND NOw()::date - activity_gmt::date < 1095 -- 3 years

group by
last_active_days

order by
last_active_days

'''

In [ ]:
df_sierra = pd.read_sql(sql=sql, con=con)

In [ ]:
# sort on the count, or y axis
source = df_sierra

alt.Chart(source).mark_bar().encode(
    
    x=alt.X('last_active_days:O'),
    y=alt.Y('count', ),
#     tooltip=['transaction_date', 'transaction_location_code', 'op_name', 'count']
).properties(
    title='TITLE',
    width=1000
).configure_title(
    # fontSize=20,
    anchor='start',
)

In [ ]:
# sort on the count, or y axis
source = df_sierra[df_sierra['last_active_days'] < 365]

alt.Chart(source).mark_bar().encode(
    
    x=alt.X('last_active_days:O'),
    y=alt.Y('count', ),
    tooltip=['last_active_days', 'count']
).properties(
    title='Patrons Last Active Days Distribution',
    width=1000
).configure_title(
    # fontSize=20,
    anchor='start',
)

In [ ]:
'''
PRESENTATION NOTE? SKIP????




What if we wanted to see the relationship between "filled hold" and "checkout"
transactions by branch ... on specific date?

'''

search_date = '2020-09-21'


# select 'checkout' and 'filled hold' transactions from all locations from 1 day ago (date(NOW() - '1 day'::interval))
sql = """
SELECT
c.id,
c.transaction_gmt,
-- c.op_code,
CASE c.op_code
    WHEN 'o' THEN 'checkout'
    WHEN 'f' THEN 'filled hold'
END as op_name,
c.patron_record_id,
c.item_record_id,
c.ptype_code,
(
	select
	m.location_code
	from
	sierra_view.statistic_group_myuser as m
	where
	m.code = c.stat_group_code_num
) as transaction_location_code

FROM

sierra_view.circ_trans as c

WHERE
c.op_code IN ('o', 'f')

AND date(c.transaction_gmt) = date(%s)

order by
c.id
"""

df = pd.read_sql(sql=sql, con=con, params=[search_date])
print(df)